# Part 4: Segment cells and nuclei.

In the following notebook you will use the Mesmer algorithm to perfom cell and nuclear segmentation.

## 1. Import packages. 
This must be done every time the notebook is started or restarted.

In [17]:
# import required packages
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import stackview
from tifffile import imread
from matplotlib import pyplot as plt
from skimage import io
from ark.segmentation import marker_quantification, segmentation_utils
from instanseg import InstanSeg
from instanseg.utils.utils import show_images

## 2. Define directory paths. 
*This must be done every time the notebook is started or restarted.

In [18]:
base_dir = "C:\\Users\\smith6jt"

In [19]:
seg_dir = os.path.join(base_dir, 'KINTSUGI', 'data', '1904_CC2B_Segmentation')
proc_dir = seg_dir.replace('_Segmentation', '_Processed')
cell_table_dir = os.path.join(seg_dir, "segmentation/cell_table")
print(f"Segmentation folder is {seg_dir}.")
print(f"Processed folder is {proc_dir}.")

Processed folder is C:\Users\smith6jt\KINTSUGI\data\1904_CC2B_Processed.


### 2.1 Create directories

This cell only needs to be run once.

In [21]:
# create directories if do not exist
for directory in [base_dir, seg_dir, proc_dir, cell_table_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

## 3. Run Instanseg

### 3.1 Segment nuclei.

In [ ]:
instanseg_fluorescence = InstanSeg("fluorescence_nuclei_and_cells", verbosity=1)
image_array_nuc, pixel_size = instanseg_fluorescence.read_image(os.path.join(proc_dir, "Fluorescence_example.tif"))
labeled_output_nuc, image_tensor_nuc  = instanseg_fluorescence.eval_medium_image(image_array_nuc, pixel_size, target= "nuclei")
display_nuc = instanseg_fluorescence.display(image_tensor_nuc, labeled_output_nuc)


### 3.2 Segment cells.

In [ ]:
image_array_cells, pixel_size = instanseg_fluorescence.read_image(os.path.join(proc_dir, "Fluorescence_example.tif"))
labeled_output_cells, image_tensor_cells  = instanseg_fluorescence.eval_medium_image(image_array_cells, pixel_size, target= "cells")
display_cells = instanseg_fluorescence.display(image_tensor_cells, labeled_output_cells)


## 4. Inspect and save segmentation results 

show_images(display_nuc, colorbar=False)

In [ ]:
show_images(display_cells, colorbar=False)

In [89]:
marker_name = "FoxP3"
marker = imread(os.path.join(proc_dir, f"{marker_name}.tif"))
dapi = imread(os.path.join(proc_dir, f"DAPI.tif"))
marker_msk = imread(os.path.join(seg_dir, f"{marker_name}.tif"))
marker_brd = imread(os.path.join(seg_dir, f"{marker_name}.tif"))
nuclear_msk = imread(os.path.join(seg_dir, f"DAPI.tif"))

x1 = 1000
x2 = 1500
y1 = 1000
y2 = 1300
stackview.curtain(marker[y1:y2, x1:x2], nuclear_msk[y1:y2, x1:x2], alpha=0.4, zoom_factor=1.6, colormap='viridis')

Save the overlaid segmentation labels for each fov (these will not display, but will save in viz_dir).

In [62]:
segmentation_utils.save_segmentation_labels(
    segmentation_dir=deepcell_output_dir,
    data_dir=deepcell_input_dir,
    output_dir=deepcell_visualization_dir,
    fovs=io_utils.remove_file_extensions(fovs),
    channels=['nuclear_channel', 'membrane_channel']
)

## 6. Visualize and save stitched segmentation masks

Define the function

In [15]:
def stitch_tiles(tiles_folder, num_tiles_width, num_tiles_height, tile_width, tile_height, output_filename):

    stitched_image = Image.new('RGB', (num_tiles_width * tile_width, num_tiles_height * tile_height))

    tile_index = 0
    
    for x in range(num_tiles_width):
        for y in range(num_tiles_height):

            left = x * tile_width
            upper = y * tile_height
            tile_path = os.path.join(tiles_folder, f'fov{tile_index}_nuclear_channel_membrane_channel_overlay.tiff')
            tile = Image.open(tile_path)
            stitched_image.paste(tile, (left, upper))

            tile_index += 1

    # Save the stitched image
    stitched_image.save(tiles_folder+output_filename)


Enter the variables.

In [ ]:
tiles_folder = '../data/LN_/segmentation/deepcell_visualization'
num_tiles_width = 7  # Adjust based on how many tiles in the width
num_tiles_height = 9  # Adjust based on how many tiles in the height
tile_width = 1500  # Width of each tile
tile_height = 1427  # Height of each tile
output_filename = "/stitched_nuc_mem.tif"

stitch_tiles(tiles_folder, num_tiles_width, num_tiles_height, tile_width, tile_height, output_filename)

Visualize the results.

In [ ]:
from tifffile import imread
image = imread(deepcell_visualization_dir+output_filename)
plt.figure(figsize = (25,25))
plt.imshow(image)

## 7. Extract segmentation data

For a full list of features extracted, please refer to the cell table section of: https://ark-analysis.readthedocs.io/en/latest/_rtd/data_types.html

In [18]:
# set to True to add nuclear cell properties to the expression matrix
nuclear_counts = True

# set to True to bypass expensive cell property calculations
# only cell label, size, and centroid will be extracted if True
fast_extraction = True

Extract the segmented imaging data to create normalized and transformed expression matrices.  Note that if you're loading your own dataset, all the imaging data must be in the same folder with each fov given its own folder and all fovs having the same channels.

In [19]:

cell_table_size_normalized, cell_table_arcsinh_transformed = \
    marker_quantification.generate_cell_table(segmentation_dir=deepcell_output_dir,
                                              tiff_dir=tiff_dir,
                                              img_sub_folder=None,
                                              fovs=fovs,
                                              extraction='center_weighting',
                                              nuclear_counts=nuclear_counts,
                                              fast_extraction=fast_extraction)

# Set the compression level if desired, ZSTD compression can offer up to a 60-70% reduction in file size.
# NOTE: Compressed `csv` files cannot be opened in Excel. They must be uncompressed beforehand.
compression = None

# Uncomment the line below to allow for compressed `csv` files.
# compression = {"method": "zstd", "level": 3}

cell_table_size_normalized.to_csv(os.path.join(cell_table_dir, 'cell_table_size_normalized.csv'),
                                  compression=compression, index=False)
cell_table_arcsinh_transformed.to_csv(os.path.join(cell_table_dir, 'cell_table_arcsinh_transformed.csv'),
                                      compression=compression, index=False)